# NAL2 WebSocket API 调用示例

这个 Notebook 演示如何通过 WebSocket API 调用 NAL2 函数。

## 使用前准备

1. 确保 WebSocket 服务器正在运行：`./scripts/start-all.sh`
2. 确保 App 已连接到服务器
3. 安装依赖：`pip install websocket-client`（只需要安装一次）

In [1]:
%pip install websocket-client

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 导入必要的库
import websocket
import json
import time
import threading
from typing import Optional, Callable

## NAL2 WebSocket 客户端类

In [3]:
class NAL2WebSocketClient:
    """NAL2 WebSocket 客户端"""
    
    def __init__(self, url: str = "ws://172.29.1.253:3000"):
        self.url = url
        self.ws: Optional[websocket.WebSocketApp] = None
        self.connected = False
        self.registered = False
        self.response_callback: Optional[Callable] = None
        
    def on_open(self, ws):
        print(f"✅ WebSocket已连接到 {self.url}")
        self.connected = True
        # 注册为Web客户端（API调用方）
        ws.send(json.dumps({"type": "register", "client": "web"}))
        
    def on_message(self, ws, message):
        try:
            data = json.loads(message)
            msg_type = data.get('type')
            
            if msg_type == 'registered':
                self.registered = True
                print("✅ 已注册为 Web 客户端")
                
            elif msg_type == 'receive_output':
                output = data.get('output')
                print("\n" + "="*60)
                print("📥 收到处理结果:")
                print(output)
                print("="*60 + "\n")
                
                if self.response_callback:
                    self.response_callback(output)
                    
            elif msg_type == 'error':
                print(f"❌ 服务器错误: {data.get('message')}")
                
        except Exception as e:
            print(f"❌ 处理消息时出错: {e}")
            
    def on_error(self, ws, error):
        print(f"❌ WebSocket错误: {error}")
        
    def on_close(self, ws, close_status_code, close_msg):
        print(f"⚠️  连接已关闭")
        self.connected = False
        self.registered = False
            
    def connect(self):
        """建立WebSocket连接"""
        print(f"🔌 正在连接到 {self.url}...")
        self.ws = websocket.WebSocketApp(
            self.url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )
        
        # 在新线程中运行
        wst = threading.Thread(target=self.ws.run_forever)
        wst.daemon = True
        wst.start()
        
        # 等待连接建立
        timeout = 5
        start_time = time.time()
        while not self.registered and time.time() - start_time < timeout:
            time.sleep(0.1)
            
        return self.registered
        
    def call_nal2_function(self, function_name: str, parameters: dict, timeout: int = 30):
        """
        调用 NAL2 函数
        
        Args:
            function_name: 函数名称
            parameters: 函数参数
            timeout: 超时时间（秒）
            
        Returns:
            处理结果字典，失败返回 None
        """
        if not self.connected or not self.registered:
            print("❌ 未连接到服务器")
            return None
            
        # 构造请求
        input_data = {
            "sequence_num": int(time.time() * 1000),
            "function": function_name,
            "input_parameters": parameters
        }
        
        input_json = json.dumps(input_data, indent=2)
        print("📤 发送请求:")
        print(input_json)
        print()
        
        # 等待结果
        result_event = threading.Event()
        result_data = {"output": None}
        
        def on_result(output):
            result_data["output"] = json.loads(output)
            result_event.set()
        
        self.response_callback = on_result
        
        # 发送请求
        message = {
            "type": "send_to_app",
            "input": input_json
        }
        self.ws.send(json.dumps(message))
        
        print("⏳ 等待App处理结果...")
        
        # 等待响应
        if result_event.wait(timeout=timeout):
            return result_data["output"]
        else:
            print("⚠️  等待超时")
            return None
    
    def close(self):
        """关闭连接"""
        if self.ws:
            self.ws.close()
            print("🔌 连接已关闭")

## 创建客户端并连接

In [4]:
# 创建客户端（修改 IP 地址为你的服务器地址）
client = NAL2WebSocketClient(url="ws://172.29.1.253:3000")

# 连接到服务器
if client.connect():
    print("\n🎉 连接成功！可以开始调用 NAL2 API")
else:
    print("\n❌ 连接失败，请检查服务器是否运行")

🔌 正在连接到 ws://172.29.1.253:3000...
✅ WebSocket已连接到 ws://172.29.1.253:3000
✅ 已注册为 Web 客户端

🎉 连接成功！可以开始调用 NAL2 API


## 调用 RealEarInsertionGain_NL2 函数

In [5]:
# 准备参数
parameters = {
    "AC": [35, 45, 40, 40, 65, 70, 70, 65, 55],
    "BC": [35, 45, 40, 40, 65, 70, 999, 999, 999],
    "L": 52,
    "limiting": 2,
    "channels": 18,
    "direction": 0,
    "mic": 1,
    "ACother": [35, 45, 40, 40, 65, 70, 70, 65, 55],
    "noOfAids": 1
}

# 调用函数
result = client.call_nal2_function(
    function_name="RealEarInsertionGain_NL2",
    parameters=parameters
)

# # 处理结果
# if result and result.get('result') == 0:
#     print("\n✅ 调用成功！")
#     reig = result['output_parameters']['REIG']
#     # print(f"\nREIG 结果 ({len(reig)} 个通道):")
#     # for i, value in enumerate(reig):
#     #     print(f"  通道 {i+1}: {value:.2f} dB")
# else:
#     print("\n❌ 调用失败")

📤 发送请求:
{
  "sequence_num": 1763110819228,
  "function": "RealEarInsertionGain_NL2",
  "input_parameters": {
    "AC": [
      35,
      45,
      40,
      40,
      65,
      70,
      70,
      65,
      55
    ],
    "BC": [
      35,
      45,
      40,
      40,
      65,
      70,
      999,
      999,
      999
    ],
    "L": 52,
    "limiting": 2,
    "channels": 18,
    "direction": 0,
    "mic": 1,
    "ACother": [
      35,
      45,
      40,
      40,
      65,
      70,
      70,
      65,
      55
    ],
    "noOfAids": 1
  }
}

⏳ 等待App处理结果...

📥 收到处理结果:
{
  "sequence_num": 1763110819228,
  "result": 0,
  "function": "RealEarInsertionGain_NL2",
  "return": 0,
  "output_parameters": {
    "REIG": [
      5.007647304598746,
      5.007647304598748,
      5.007647304598748,
      5.00764730459875,
      8.782628543120119,
      12.817052758575704,
      14.003896630637604,
      16.4700814555311,
      19.695092380391845,
      22.854885817450874,
      23.963659016703648

## 完成：关闭连接

In [6]:
# 使用完毕后关闭连接
client.close()
print("\n👋 Demo 完成！")

⚠️  连接已关闭
🔌 连接已关闭

👋 Demo 完成！


## 总结

这个 Notebook 演示了：

1. ✅ 连接到 NAL2 WebSocket API
2. ✅ 调用 RealEarInsertionGain_NL2 函数
3. ✅ 处理和显示结果
4. ✅ 批量处理多组参数
5. ✅ 可视化结果（可选）

### 下一步

- 修改参数进行更多测试
- 集成到你自己的应用中
- 参考 README.md 了解更多高级用法